In [1]:
import pandas as pd
import numpy as np
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [4]:
df = pd.read_csv('./Genomes-GeneticDisorders/train_processed_encoded.csv',index_col=0)
# df.drop(['Test 1','Test 2','Test 3','Test 4','Test 5'], axis=1, inplace=True )
df.head()

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),...,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
0,2.0,1.0,0.0,1.0,0.0,4.760603,5.0,20.0,0.0,0.0,...,1.0,9.857562,3.0,1.0,1.0,1.0,1.0,1.0,0.0,5.0
1,4.0,1.0,1.0,0.0,0.0,4.910669,5.0,23.0,1.0,1.0,...,0.0,5.522560,2.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0
2,6.0,1.0,0.0,0.0,0.0,4.893297,23.0,22.0,0.0,0.0,...,1.0,7.477132,2.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0
3,12.0,1.0,0.0,1.0,0.0,4.705280,3.0,20.0,1.0,1.0,...,1.0,7.919321,1.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0
4,11.0,1.0,0.0,1.0,1.0,4.720703,14.0,20.0,0.0,1.0,...,0.0,4.098210,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [5]:
# Define X data
X = df.iloc[:,0:28]
X = (X - X.mean())/X.std()
X.head()

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),...,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5
0,-1.127531,0.82473,-0.801981,0.800610,-0.874915,-0.692508,-0.850755,-1.081659,-0.996226,-0.898872,...,0.895583,-0.002060,0.905726,0.940970,1.162410,0.762643,0.821633,0.850322,1.105249,1.182666
1,-0.655954,0.82473,1.246856,-1.248992,-0.874915,0.059089,-0.850755,-0.879060,1.003743,1.112455,...,0.895583,-0.002060,-1.104036,-0.778559,0.294317,0.762643,0.821633,0.850322,1.105249,-0.845509
2,-0.184377,0.82473,-0.801981,-1.248992,-0.874915,-0.027917,0.977537,-0.946593,-0.996226,-0.898872,...,0.895583,1.489342,0.905726,-0.003256,0.294317,-1.311171,0.821633,0.850322,1.105249,1.182666
3,1.230354,0.82473,-0.801981,0.800610,-0.874915,-0.969588,-1.053899,-1.081659,1.003743,1.112455,...,0.895583,-0.747761,0.905726,0.172144,-0.573775,-1.311171,-1.217033,0.850322,-0.904733,-0.845509
4,0.994566,0.82473,-0.801981,0.800610,1.142917,-0.892347,0.063391,-1.081659,-0.996226,1.112455,...,-1.116540,1.489342,-1.104036,-1.343544,1.162410,-1.311171,-1.217033,-1.175972,-0.904733,-0.845509


In [6]:
# Define Encoder
encoder = LabelEncoder()

# Define Labels
GD = pd.get_dummies(df["Genetic Disorder"])
DS = pd.get_dummies(df["Disorder Subclass"])
GDDS = df["Genetic Disorder"].astype(str) + df["Disorder Subclass"].astype(str)
encoder.fit(GDDS)
GDDS = pd.get_dummies(pd.Series(encoder.transform(GDDS))) #0.06.0

In [7]:
labelsets = [GD,DS,GDDS]
datanames = ['GD', 'DS', 'GDDS']
data = {}

In [8]:
for i in range(len(labelsets)) :
    
    # Split data
    X_train, X_rem, y_train, y_rem = train_test_split(X,labelsets[i], train_size=0.7)
    X_val, X_test, y_val, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
    
    print('-'*9, datanames[i], '-'*9)
    print(X_train.shape), print(y_train.shape)
    print(X_val.shape), print(y_val.shape)
    print(X_test.shape), print(y_test.shape)
    
    data[datanames[i]] = {
        'X': {
            'train' : X_train,
            'val' : X_val,
            'test' : X_test
        },
        'y' : {
            'train': y_train,
            'val': y_val,
            'test': y_test
        }
    }

--------- GD ---------
(15458, 28)
(15458, 3)
(3312, 28)
(3312, 3)
(3313, 28)
(3313, 3)
--------- DS ---------
(15458, 28)
(15458, 9)
(3312, 28)
(3312, 9)
(3313, 28)
(3313, 9)
--------- GDDS ---------
(15458, 28)
(15458, 17)
(3312, 28)
(3312, 17)
(3313, 28)
(3313, 17)


In [9]:
def evaluate(y_pred, y_true) :
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(np.array(y_true), axis=1)
    
   # Performance metrics
    accuracy = np.sum(y_pred == y_true) / len(y_pred)
    print('Accuracy:', accuracy)
    
    # Confusion matrix
    confmat = confusion_matrix(y_true, y_pred)
    print(confmat)
#     tn, fp, fn, tp = confmat.ravel()

    print(classification_report(y_true, y_pred))
#     print('Specificty: '+ str(tn / (tn+fp)))
#     disp = ConfusionMatrixDisplay(confusion_matrix=confmat,display_labels=best.classes_)  
#     disp.plot()
#     plt.show()

In [12]:
# define baseline model:
# input: X data
# output: GD and DS

# Define callback for early stopping
callback = EarlyStopping(monitor='loss', min_delta=0.005,patience=3)

# Define model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(24, input_dim=28, activation='relu'))
    model.add(Dense(18, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 
baseline_model = baseline_model()
baseline_model.fit(
                   data['GD']['X']['train'], 
                   data['GD']['y']['train'], 
                   validation_data=(
                   data['GD']['X']['val'],
                   data['GD']['y']['val']),
                   epochs=50,
                   callbacks=[callback]
                  )
y_pred = baseline_model.predict(data['GD']['X']['test'])
evaluate(y_pred, data['GD']['y']['test'])

Epoch 1/50
484/484 [==============================] - 0s 582us/step - loss: 0.9664 - accuracy: 0.4898 - val_loss: 0.8665 - val_accuracy: 0.5528
Epoch 2/50
484/484 [==============================] - 0s 363us/step - loss: 0.8452 - accuracy: 0.5658 - val_loss: 0.8420 - val_accuracy: 0.5734
Epoch 3/50
484/484 [==============================] - 0s 360us/step - loss: 0.8249 - accuracy: 0.5861 - val_loss: 0.8212 - val_accuracy: 0.5897
Epoch 4/50
484/484 [==============================] - 0s 380us/step - loss: 0.8003 - accuracy: 0.6038 - val_loss: 0.8106 - val_accuracy: 0.5972
Epoch 5/50
484/484 [==============================] - 0s 373us/step - loss: 0.7791 - accuracy: 0.6166 - val_loss: 0.8044 - val_accuracy: 0.5927
Epoch 6/50
484/484 [==============================] - 0s 366us/step - loss: 0.7852 - accuracy: 0.6099 - val_loss: 0.8000 - val_accuracy: 0.6081
Epoch 7/50
484/484 [==============================] - 0s 356us/step - loss: 0.7744 - accuracy: 0.6172 - val_loss: 0.7969 - val_accuracy:

In [13]:
np.argmax(y_pred, axis=1)

array([ 6, 13, 14, ..., 13,  6, 13])

In [14]:
data['GDDS']['y']['test']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
10376,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
15696,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2171,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
881,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10056,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14134,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1512,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
13029,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4768,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# define baseline model
def GD_model():
    # create model
    model = Sequential()
    model.add(Dense(14, input_dim=28, activation='relu'))
    model.add(Dense(14, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 
GD_model = GD_model()
GD_model.fit(X, Y_GD, epochs=15)
# estimator = KerasClassifier(build_fn=GD_model, epochs=20, batch_size=25, verbose=0)
# kfold = KFold(n_splits=2, shuffle=True)
# results = cross_val_score(estimator, X, Y_GD, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


In [ ]:
# define disease subclass model
def DS_model():
    # create model
    model = Sequential()
    model.add(Dense(14, input_dim=31, activation='relu'))
    model.add(Dense(14, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 
DS_model = DS_model()
DS_model.fit(X2, Y_DS, epochs=15)

In [ ]:
# define baseline model
def DS_model2():
    # create model
    model = Sequential()
    model.add(Dense(28, input_dim=31, activation='relu'))
    model.add(Dense(23, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 
DS_model2 = DS_model2()
DS_model2.fit(X2, Y_DS, epochs=25)

In [ ]:
test = np.array(X.iloc[0:3,:])
GD_model.predict(test)

In [ ]:
X2 = X.join(Y_GD)
# Standardize
X2 = (X2 - X2.mean())/X2.std()

In [ ]:
X2.shape

In [ ]:
GD_out = GD_model.predict(X)
print(np.sum(np.argmax(GD_out, axis=1) == np.argmax(np.array(Y_GD),axis=1)) / 22083)

In [ ]:
X3 = X.join(pd.DataFrame(GD_out))
X3.shape

In [ ]:
np.argmax(np.array(Y_comb),axis=1)

In [ ]:
outputs2 = DS_model.predict(X3)
print(np.sum(np.argmax(outputs2, axis=1) == np.argmax(np.array(Y_comb),axis=1)))

In [ ]:
M1_pred = GD_model.predict(test)
print(np.argmax(M1_pred, axis=1))
print(np.argmax(np.array(Y_GD)[0:3,:], axis=1))


In [ ]:
X_ex = np.concatenate((test, M1_pred), axis=1)
M2_pred = DS_model.predict(X_ex)
print(np.argmax(M2_pred, axis=1))
print(np.argmax(np.array(Y_DS)[0:3,:], axis=1))

In [ ]:
baseline_pred = baseline_model.predict(X.iloc[0:3,:])
print(np.argmax(baseline_pred, axis=1))
print(np.argmax(np.array(Y_comb)[0:3,:], axis=1))

In [ ]:
encoder.inverse_transform(np.argmax(baseline_pred, axis=1))

In [ ]:
encoder.inverse_transform(np.argmax(np.array(Y_comb)[0:3,:], axis=1))